Author: Jonathan Gong

In [7]:
import pandas as pd
import os
import sys
from sktime.annotation.hmm_learn import GMMHMM 
import numpy as np

# Data processing

In [16]:
csv_files = []
for file in os.listdir('../data/raw'):
    if file.endswith('.csv'):
        csv_files.append(pd.read_csv(f'../data/raw/{file}'))

/var/folders/22/0fj82gz10j5cvdcxtqcw5yc80000gn/T/ipykernel_61427/895166927.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_files.append(pd.read_csv(f'../data/raw/{file}'))


In [19]:
each_bgl =[]
for file in csv_files:
    each_df = file[["date", "bgl"]]
    each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)
    each_bgl.append(each_df.dropna(subset=each_df.columns[:4], how='all'))

/var/folders/22/0fj82gz10j5cvdcxtqcw5yc80000gn/T/ipykernel_61427/872985159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)
/var/folders/22/0fj82gz10j5cvdcxtqcw5yc80000gn/T/ipykernel_61427/872985159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)


In [20]:
days_patients = []
for bgl in each_bgl:
    # (hour:minute:second)
    bgl['time'] = bgl['date'].dt.time
    # (year-month-day)
    bgl['day'] = bgl['date'].dt.date
    # unique days
    unique_days = bgl['day'].unique()
    day_data = []
    for i, day in enumerate(unique_days):
        bgl_day = bgl[bgl['day'] == day]
        day_data.append(bgl_day["bgl"])

    days_patients.append(day_data)

In [29]:
sequences = days_patients[0]
sequences = np.concatenate(sequences)
sequences = sequences[~np.isnan(sequences)]

In [30]:
sequences


array([ 98., 100.,  98., ...,  89.,  88.,  88.])

In [31]:
model = GMMHMM(n_components=2, n_mix=2) 
model = model.fit(sequences) 

Model is not converging.  Current: -183579.8275463407 is not greater than -178807.85189297324. Delta is -4771.975653367466
